In [1]:
import torch
import matplotlib.pyplot as plt
from torchvision import datasets,transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 60.8MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 1.98MB/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 12.6MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.49MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
train_loader = DataLoader(train_data, batch_size=4, shuffle=True)
data=iter(train_loader)
test_data,test_label=next(data)

In [4]:
test_data.shape,test_label.shape

(torch.Size([4, 1, 28, 28]), torch.Size([4]))

In [5]:
def cmp(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [6]:
# conv=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=3,stride=1)
# maxpool=nn.MaxPool2d(kernel_size=2,stride=2)
# fc1=nn.Linear(in_features=1014,out_features=500)
# fc2=nn.Linear(in_features=500,out_features=10)

# x=F.relu(conv(test_data))
# x=maxpool(x)
# x=torch.flatten(x,start_dim=0)
# x=F.relu(fc1(x))
# x=fc2(x)

In [7]:
# out=conv(test_data)

In [8]:
# out.shape

In [9]:
# relu_out=F.relu(out)
# relu_out.shape

In [10]:
# maxpool=nn.MaxPool2d(kernel_size=2,stride=2)
# pool_out=maxpool(relu_out)
# pool_out.shape

In [11]:
# flatten_out=torch.flatten(pool_out,start_dim=1)
# flatten_out.shape

In [12]:
# fc1=nn.Linear(in_features=1014,out_features=10)
# fc1_out=fc1(flatten_out)
# fc1_out.shape

In [13]:
def relu(z):
  return torch.where(z<=0,0,z)

In [14]:
def softmax(z):
  return(torch.exp(z)/(torch.exp(z).sum(dim=1,keepdim=True)))

# Manually Backpropagation on 1 Image

In [15]:
test_data.shape

torch.Size([4, 1, 28, 28])

In [16]:
# for 2D Convolution

in_channels=1
out_channels=1
kernel_size=(3,3)
stride=(1,1)

batch_size,image_channels,input_dim_hight,input_dim_width=test_data.shape
# input_dim_hight=test_data.shape[1]
# input_dim_width=test_data.shape[2]
kernel_size_hight=kernel_size[0]
kernel_size_width=kernel_size[1]
stride_hight=stride[0]
stride_width=stride[1]

filters=torch.randn(out_channels,in_channels,kernel_size_hight,kernel_size_width)*0.01
filters.requires_grad_(True)

conv_out_dim_hight=((input_dim_hight-kernel_size_hight)//stride_hight)+1
conv_out_dim_width=((input_dim_width-kernel_size_width)//stride_width)+1

conv_out=torch.zeros(batch_size,out_channels,conv_out_dim_hight,conv_out_dim_width)

for b in range(batch_size):
  for i in range(filters.shape[0]):
    y_tmp=y_out=0
    while y_tmp+kernel_size_hight <= input_dim_hight:
      x_tmp=x_out=0
      while x_tmp+kernel_size_width <= input_dim_width:
        patch=test_data[b,:,y_tmp:y_tmp+kernel_size_hight,x_tmp:x_tmp+kernel_size_width]
        conv_out[b,i,y_out,x_out]=torch.sum(filters[i]*patch)
        x_tmp += stride_width
        x_out +=1
      y_tmp +=stride_hight
      y_out +=1

In [17]:
conv_out.shape

torch.Size([4, 1, 26, 26])

In [18]:
relu_conv_out=relu(conv_out)

In [19]:
relu_conv_out.shape

torch.Size([4, 1, 26, 26])

In [20]:
# 2D Max-Pooling

pool_kernel_size=(2,2)
pool_stride=(2,2)

pool_kernel_size_height=pool_kernel_size[0]
pool_kernel_size_width=pool_kernel_size[1]
pool_stride_height=pool_stride[0]
pool_stride_width=pool_stride[1]

batch_size,num_channel,height,width=relu_conv_out.shape

pool_out_dim_height=((height-pool_kernel_size_height)//pool_stride_height)+1
pool_out_dim_width=((width-pool_kernel_size_width)//pool_stride_width)+1

pool_conv_out=torch.zeros(batch_size,num_channel,pool_out_dim_height,pool_out_dim_width)




for b in range(batch_size):
  for i in range(num_channel):
    y_tmp=y_out=0
    while y_tmp+pool_kernel_size_height <= height:
      x_tmp=x_out=0
      while x_tmp+pool_kernel_size_width <= width:
        patch=relu_conv_out[b,i,y_tmp:y_tmp+pool_kernel_size_height,x_tmp:x_tmp+pool_kernel_size_width]

        pool_conv_out[b,i,y_out,x_out]=torch.max(patch)
        x_tmp += pool_stride_width
        x_out +=1
      y_tmp += pool_stride_height
      y_out +=1

In [21]:
pool_conv_out.shape

torch.Size([4, 1, 13, 13])

In [22]:
flatten_conv_out=torch.flatten(pool_conv_out,start_dim=1)
flatten_conv_out.shape

torch.Size([4, 169])

In [23]:
w1=torch.randn(flatten_conv_out.shape[1],10)*0.01
w1.requires_grad_(True)
b1=torch.randn(1,10)*0.01
b1.requires_grad_(True)


tensor([[ 0.0145,  0.0081,  0.0008,  0.0137, -0.0010, -0.0025, -0.0027,  0.0182,
         -0.0016, -0.0009]], requires_grad=True)

In [24]:
z1=flatten_conv_out@w1+b1

z1_max=z1.max(1,keepdim=True).values
norm_z1=z1-z1_max
exp_z1=torch.exp(norm_z1)
exp_sum=(exp_z1.sum(dim=1,keepdim=True))
exp_sum_inv=exp_sum**-1
probs=exp_z1*exp_sum_inv
logprobs=torch.log(probs)
loss=-logprobs[range(len(test_data)),test_label].mean()

In [25]:
z1.shape

torch.Size([4, 10])

In [26]:
w1.grad=None
b1.grad=None

for i in [conv_out,patch,relu_conv_out,pool_conv_out,z1,z1_max,norm_z1,exp_z1,exp_sum,probs,logprobs,exp_sum_inv,flatten_conv_out]:
  i.retain_grad()
loss.backward()

In [27]:
dlogprobs=torch.zeros_like(logprobs)
dlogprobs[range(len(test_data)),test_label]=-1.0/len(test_data)
dprobs=(1.0/probs)*dlogprobs
dexp_sum_inv=(exp_z1*dprobs).sum(1,keepdim=True)
dexp_z1=exp_sum_inv*dprobs
dexp_sum=(-exp_sum**-2)*dexp_sum_inv
dexp_z1+=torch.ones_like(exp_z1)*dexp_sum
dnorm_z1=torch.exp(norm_z1)*dexp_z1
dz1=dnorm_z1.clone()
dz1_max=(-dnorm_z1).sum(1,keepdim=True)
dz1+=torch.nn.functional.one_hot(z1.max(1).indices,num_classes=z1.shape[1])*dz1_max
dw1=flatten_conv_out.T@dz1
db1=dz1.sum(0)
dflatten_conv_out=dz1@w1.T

dpool_conv_out=dflatten_conv_out.reshape(pool_conv_out.shape)
drelu_conv_out=torch.zeros_like(relu_conv_out)


for b in range(batch_size):
  for i in range(num_channel):
    y_tmp=y_out=0
    while y_tmp+pool_kernel_size_height <= height:
      x_tmp=x_out=0
      while x_tmp+pool_kernel_size_width <= width:
        patch=relu_conv_out[b,i,y_tmp:y_tmp+pool_kernel_size_height,x_tmp:x_tmp+pool_kernel_size_width]
        max_value=torch.max(patch)
        positions = torch.nonzero(patch == max_value, as_tuple=False)
        positions_list = [tuple(pos.tolist()) for pos in positions]
        for p in positions_list:
          drelu_conv_out[b,i,y_tmp+p[0],x_tmp+p[1]]+=dpool_conv_out[b,i,y_out,x_out]/len(positions_list)
        x_tmp += pool_stride_width
        x_out +=1
      y_tmp += pool_stride_height
      y_out +=1

dconv_out=conv_out.clone()
dconv_out=torch.where(dconv_out<=0,0,1)*drelu_conv_out



dfilters=torch.zeros_like(filters)
batch_size,channel,kernel_size_hight,kernel_size_width=dconv_out.shape
for b in range(batch_size):
  for i in range(filters.shape[0]):
    for j in range(1):
      y_tmp=y_out=0
      while y_tmp+kernel_size_hight <= input_dim_hight:
        x_tmp=x_out=0
        while x_tmp+kernel_size_width <= input_dim_width:
          patch=test_data[b,:,y_tmp:y_tmp+kernel_size_hight,x_tmp:x_tmp+kernel_size_width]
          dfilters[i,j,y_out,x_out]+=torch.sum(dconv_out[b]*patch)
          x_tmp += stride_width
          x_out +=1
        y_tmp +=stride_hight
        y_out +=1


print("From Loss to Fully Connected Layer")
cmp("dlogprobs",dlogprobs,logprobs)
cmp("dprobs",dprobs,probs)
cmp("dexp_sum_inv",dexp_sum_inv,exp_sum_inv)
cmp("dexp_sum",dexp_sum,exp_sum)
cmp("dexp_z1",dexp_z1,exp_z1)
cmp("dnorm_z1",dnorm_z1,norm_z1)
cmp("dz1_max",dz1_max,z1_max)
cmp("dz1",dz1,z1)
cmp("dw1",dw1,w1)
cmp("db1",db1,b1)
cmp("dflatten_conv_out",dflatten_conv_out,flatten_conv_out)

print()
print("From Fully Connected Layer to ReLu Activation")
cmp("dpool_conv_out",dpool_conv_out,pool_conv_out)
cmp("drelu_conv_out",drelu_conv_out,relu_conv_out)
cmp("dconv_out",dconv_out,conv_out)
cmp("dfilters",dfilters,filters)

From Loss to Fully Connected Layer
dlogprobs       | exact: True  | approximate: True  | maxdiff: 0.0
dprobs          | exact: True  | approximate: True  | maxdiff: 0.0
dexp_sum_inv    | exact: True  | approximate: True  | maxdiff: 0.0
dexp_sum        | exact: True  | approximate: True  | maxdiff: 0.0
dexp_z1         | exact: True  | approximate: True  | maxdiff: 0.0
dnorm_z1        | exact: True  | approximate: True  | maxdiff: 0.0
dz1_max         | exact: True  | approximate: True  | maxdiff: 0.0
dz1             | exact: True  | approximate: True  | maxdiff: 0.0
dw1             | exact: True  | approximate: True  | maxdiff: 0.0
db1             | exact: True  | approximate: True  | maxdiff: 0.0
dflatten_conv_out | exact: True  | approximate: True  | maxdiff: 0.0

From Fully Connected Layer to ReLu Activation
dpool_conv_out  | exact: True  | approximate: True  | maxdiff: 0.0
drelu_conv_out  | exact: True  | approximate: True  | maxdiff: 0.0
dconv_out       | exact: True  | approximate: